In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from basic.claims import connect #, get_claims, get_users, get_kibana_gate, products_bankiros, nulls, get_medium, get_claims_raw
# from basic.external import cpa_parse, get_yandex_data
# from basic.dscore import cre_get, mfo_score
from basic.reports import cre_keyword_report, master_report, registrations_medium
from basic.vars import regions_mapper, claim_offers
from bson.objectid import ObjectId
import requests
import json
from datetime import datetime, date, timedelta
import pandas as pd
from time import sleep
from base64 import b64encode
import numpy as np
# from robobrowser import RoboBrowser
import re
# import csv, sqlite3
# import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
from calendar import monthrange
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1000)
# from IPython.display import Filelink, FileLinks


client, es = connect()

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Доспуп к Эластику 	 True
Доспуп к Монге 		 True


In [2]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:startURL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][2]["name"],
            #'area':result[i]["dimensions"][2]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            #'interest':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1




INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [3]:
df['visits'].sum()

2746.0

In [5]:
df['date']  =pd.to_datetime(df['date'],format='%Y-%m-%d')

In [6]:
from datetime import datetime
def income_group(date):
    if datetime(2021,7,22) <= date <= datetime(2021,8,20) : return 'Июльская рассылка'
    if datetime(2021,10,25)<= date <= datetime(2022,5,16): return 'Октябрьская рассылка'
#     if datetime(2021,5,17) <= date <= datetime(2021,6,20): return 'Майская рассылка'
#     if datetime(2021,6,21) <= date<= datetime(2021,7,19): return'Июньская рассылка'
#     if datetime(2021,7,20) <= date <= datetime(2021,8,20): return'Июльская рассылка'
#     if datetime(2021,8,21) <= date <= datetime(2021,9,25): return'Августcкая рассылка'
#     if datetime(2021,9,26) <= date <= datetime(2021,10,18): return'Сентябрьская рассылка'
#     if datetime(2021,10,19) <= date <= datetime(2021,11,20): return'Октябрьская рассылка'


    else: 
        return ''
    
df['income_group_first'] = df['date'].apply(income_group)
df

,date,visits,week,income_group_first
0,2021-07-12,2.0,29,
1,2021-07-13,1.0,29,
2,2021-07-14,2.0,29,
3,2021-07-15,1.0,29,
4,2021-07-19,2.0,30,
5,2021-07-20,1.0,30,
6,2021-07-23,504.0,30,Июльская рассылка
7,2021-07-24,414.0,30,Июльская рассылка
8,2021-07-25,89.0,30,Июльская рассылка
9,2021-07-26,73.0,31,Июльская рассылка


In [7]:
from datetime import datetime
def income_group(date):
    if datetime(2021,7,22) <= date <= datetime(2021,8,20) : return 'Июльская рассылка'
    if datetime(2021,10,25)<= date <= datetime(2022,5,16): return 'Октябрьская рассылка'
#     if datetime(2021,5,17) <= date <= datetime(2021,6,20): return 'Майская рассылка'
#     if datetime(2021,6,21) <= date<= datetime(2021,7,19): return'Июньская рассылка'
#     if datetime(2021,7,20) <= date <= datetime(2021,8,20): return'Июльская рассылка'
#     if datetime(2021,8,21) <= date <= datetime(2021,9,25): return'Августcкая рассылка'
#     if datetime(2021,9,26) <= date <= datetime(2021,10,18): return'Сентябрьская рассылка'
#     if datetime(2021,10,19) <= date <= datetime(2021,11,20): return'Октябрьская рассылка'



    else: 
        return 'не определено'
    
df['income_group_dt'] = df['date'].apply(income_group)

def income_group_number(income_group_dt):
#     if income_group_dt=='Мартовская рассылка' : return 1
#     if income_group_dt=='Апрельская рассылка' : return 2
#     if income_group_dt=='Майская рассылка' : return 3
#     if income_group_dt=='Июньская рассылка' : return 4
    if income_group_dt=='Июльская рассылка' : return 1
#     if income_group_dt=='Августcкая рассылка' : return 6
#     if income_group_dt=='Сентябрьская рассылка' : return 7
    if income_group_dt=='Октябрьская рассылка' : return 2



    
   


    else: 
        return 'не определено'
    
df['income_group_number'] = df['income_group_dt'].apply(income_group_number)
df
df.to_excel('Tele2 KK/Теле2_мз_траф.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(0) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_траф.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})

df1.to_excel('Tele2 KK/Теле2_мз_траф2.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(1) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_траф2.xlsx')
set_with_dataframe(worksheet, df1)


In [8]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date,ym:s:regionArea",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:startURL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][2]["name"],
            'area':result[i]["dimensions"][1]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            #'interest':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df['income_group_first'] = df['date'].apply(income_group)
df = df.groupby(['income_group_first','area'],as_index=False).agg({'visits':'sum'})
df
df.to_excel('Tele2 KK/Теле2_мз_гор.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(2) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_гор.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})

df1.to_excel('Tele2 KK/Теле2_мз_гор.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(3) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_гор.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [9]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date,ym:s:gender",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:startURL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            'gender':result[i]["dimensions"][1]["name"],
            #'area':result[i]["dimensions"][1]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            #'interest':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df['income_group_first'] = df['date'].apply(income_group)
df = df.groupby(['income_group_first','gender'],as_index=False).agg({'visits':'sum'})
df
df.to_excel('Tele2 KK/Теле2_мз_пол.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(4) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_пол.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})

df1.to_excel('Tele2 KK/Теле2_мз_пол2.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(5) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_пол2.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [10]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date,ym:s:interest",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:startURL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][1]["name"],
            #'area':result[i]["dimensions"][1]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            'interest':result[i]["dimensions"][1]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df['income_group_first'] = df['date'].apply(income_group)
df = df.groupby(['income_group_first','interest'],as_index=False).agg({'visits':'sum'})
df
df.to_excel('Tele2 KK/Теле2_мз_инт.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(6) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_инт.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})

df1.to_excel('Tele2 KK/Теле2_мз_инт2.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(7) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_инт2.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [11]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date,ym:s:interest2d1,ym:s:interest2d2",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:startURL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][1]["name"],
            'interest2':result[i]["dimensions"][2]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            'interest':result[i]["dimensions"][1]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df = df[df['interest']=='Finance']
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df['income_group_first'] = df['date'].apply(income_group)
df = df.groupby(['income_group_first','interest','interest2'],as_index=False).agg({'visits':'sum'})
df
df.to_excel('Tele2 KK/Теле2_мз_инте.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(8) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_инте.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})
df1.to_excel('Tele2 KK/Теле2_мз_инте2.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(9) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_инте2.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:58: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [12]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date,ym:s:ageInterval",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:startURL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][1]["name"],
            #'interest2':result[i]["dimensions"][2]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            #'interest':result[i]["dimensions"][1]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
#df = df[df['interest']=='Finance']
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df['income_group_first'] = df['date'].apply(income_group)
df = df.groupby(['income_group_first','age_group'],as_index=False).agg({'visits':'sum'})
df
df.to_excel('Tele2 KK/Теле2_мз_воз.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(10) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_воз.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})

df1.to_excel('Tele2 KK/Теле2_мз_воз2.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(11) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_воз2.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:58: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [13]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date,ym:s:mobilePhone",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:startURL=='https://t2.odobrim.ru/'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'phone':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][1]["name"],
            #'interest2':result[i]["dimensions"][2]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            #'interest':result[i]["dimensions"][1]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
#df = df[df['interest']=='Finance']
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df['income_group_first'] = df['date'].apply(income_group)
df = df.groupby(['income_group_first','phone'],as_index=False).agg({'visits':'sum'})
df
df.to_excel('Tele2 KK/Теле2_мз_дев.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(12) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_дев.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})

df1.to_excel('Tele2 KK/Теле2_мз_дев2.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(13) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_мз_дев2.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:58: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [14]:
df

,income_group_first,phone,visits
0,Июльская рассылка,ASUS,9.0
1,Июльская рассылка,Alcatel,8.0
2,Июльская рассылка,Apple,171.0
3,Июльская рассылка,BQ,29.0
4,Июльская рассылка,Blackview,3.0
5,Июльская рассылка,CUBE,1.0
6,Июльская рассылка,DEXP,1.0
7,Июльская рассылка,Digma,2.0
8,Июльская рассылка,FinePower,1.0
9,Июльская рассылка,Fly,2.0


In [15]:
since = date(2021,7,1)
till = date(2021,12,31)

In [16]:
def get_kibana_reg(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 1000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/1000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-1000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID','currentPage',
#                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
#                   'transaction_id', 'clickId', 
                'sourceGlobal']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

regs = get_kibana_reg(es, since,till)


INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.212s]


12635; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.099s]


11635; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.031s]


...

2635; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.069s]


1635; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.024s]


635; 

In [17]:
regs = regs[regs['utm_source']=='t2_kk']
regs['dt_reg'] = pd.to_datetime(regs['dt_reg'])
regs['date']=regs['dt_reg']
regs['income_group_first'] = regs['date'].apply(income_group)
df = regs.groupby('income_group_first',as_index=False).agg({'userId':'count'})
df.to_excel('Tele2 KK/Теле2_рег.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(14) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_рег.xlsx')
set_with_dataframe(worksheet, df1)

In [18]:
# !pip install gspread_dataframe

def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)    
            
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)  # сюда json файл если он не в той же папке
            creds = flow.run_local_server(port=0)        
            
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)    
    return creds

def pull_sheet_data(SCOPES,SPREADSHEET_ID,RANGE_NAME):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=RANGE_NAME).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data
    
def to_float(x):
    try:
        return(float(x.replace("\xa0", '').replace(" ", '').strip()))
    except Exception as e:
        return np.nan

def cre_get(es, since):
    
    chunk = 10000
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    payload = {"size": chunk,
               "_source": ["params.parsed"],
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword": "CRE"}},
                         {"range":{"timestamp":{"gte":gte,"format":"epoch_millis"}}}
                        ]}},
"_source": ["params.parsed", "timestamp"]
              }
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=3000)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/chunk)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=3000)
        hits = hits + d['hits']['hits']
        print(doc_count-chunk*i, end ="; ", flush=True)
        sleep(0.5)
#         break
    print(len(hits), 'raz')
    cre = []
    for h in hits:
        if len(h['_source']['params']['parsed'])>10:
            h2 = json.loads(h['_source']['params']['parsed'].replace("'", '"'))
            h2.update({"timestamp": pd.to_datetime(h['_source']['timestamp'])})
            cre.append(h2)
    df = pd.DataFrame(cre).sort_values("timestamp", 
                                       ascending=False).drop_duplicates('userId', keep='first').reset_index(drop=True)
    df.index = df['userId']
      
            
    return df['score'].to_dict()

def mm_requestId(x):
    if type(x)==dict and 'NOTE' in x.keys():
        if x['NOTE'].find('HTTP_RESPONSE_BODY') >0:
            htr = json.loads(x['NOTE'][x['NOTE'].find('HTTP_RESPONSE_BODY')+20:])
            htr = htr['requestId'] if 'requestId' in htr.keys() else np.nan
#             print(htr)
        else: 
            htr = np.nan
    else: 
        htr = np.nan
    return htr

def get_exact_claims(since, till, client):
    req = {"createdAt": {"$gte": datetime.combine(since, datetime.min.time()) - timedelta(hours=3),
                          "$lt": datetime.combine(till, datetime.min.time())- timedelta(hours=3)},
#            "product._id": {"$in": [ObjectId("5e9846be79dd27000141f476"),
# ObjectId("5ef4473acf22a90001010445")]},
           #"product._id": {"$in": [ObjectId(product) for product in products]}}
            "status": {"$nin": [ "ERROR", "NEW"]}} 
    results_count = client['claim-service']['claims'].count_documents(req)
    # print(req)
    print(results_count)
    all_ans = []
    batch_size = 10000
    for i in range(results_count//batch_size+1):
        ans = list(client['claim-service']['claims'].find(req, {"product": 1, "partner":1,
                                                                "borrower": 1, "statusFields":1,
                                                                "status": 1, "adapterName": 1,
                                                                "createdAt": 1, 'type':1,
                                                                "updatedAt":1, "clickId": 1,
                                                                "whence":1}).limit(batch_size).skip(i*batch_size))
        all_ans = all_ans + ans
    #ans = trans(all_ans, client)
    df = pd.DataFrame(all_ans)
    df['id'] = df["_id"].apply(lambda x: str(x))
    df['dt'] = df["createdAt"].apply(lambda x: (x+timedelta(hours=3)).date())
    df['productId'] = df['product'].apply(lambda x: str(x['_id']))
    df['product_type'] = df['product'].apply(lambda x: x['productType'] if 'productType' in x.keys() else np.nan)
    df['product_name'] = df['product'].apply(lambda x: x['name'])
    df['partner'] = df['partner'].apply(lambda x: x['name'])
    df['userId'] = df['borrower'].apply(lambda x: x['userId'])
    df['phone'] = df['borrower'].apply(lambda x: x['phone'] if 'phone' in x.keys() else np.nan)

    return df


def get_kibana_claims(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"CLAIM"}},
#                          {"match_phrase":{"params.sourceGlobal.keyword":{"query": sourceGlobal }}},
                         {"range":{"timestamp":{"gte":gte,"format":"epoch_millis"}}}
                        ]}},
"_source": ["params.parsed",'params.clickId', 'params.userId', 'params.utm_campaign', 'params.utm_medium', 
            'params.utm_content','params.utm_source', 'params.utm_term', 'params.gaId', 'params.yandexClientID',
            'params.sourceGlobal',"timestamp"]}
              
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
                  'transaction_id', 'clickId', 'sourceGlobal', 'utm_content','orderId','transactionId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

def get_kibana_postback(es, since):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(date.today()+timedelta(days=1), datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 10000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"POSTBACK"}},
#                          {"match_phrase":{"params.whence.keyword":{"query": whence }}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/10000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-10000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', 'payout', 'status',
                'transaction_id', 'externalId','orderId','transactionId']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['date'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

In [19]:
df = get_exact_claims(since, till, client)
df['requestId'] = df[df['partner']=='MoneyMan']['statusFields'].apply(mm_requestId)
dpbk = get_kibana_postback(es, since)
def float_apply(x):
    try:
        return float(x)
    except ValueError:
        return None
dpbk['payout'] = dpbk['payout'].apply(float_apply)
dpbk['payout'] = dpbk.apply(lambda x: x['payout']*1.2 if x['partnerName']=='admitad' else x['payout'], axis=1)
dpbk = dpbk[dpbk['status'].isin(['APPROVED', 'approved', 'ISSUED'])].sort_values('payout').drop_duplicates('transitionId', keep='last')
dr = pd.merge(left=df, right=dpbk, left_on='id', right_on='transitionId', how='left')
dc = get_kibana_claims(es, since,till)[['clickId', 'userId', 'utm_campaign', 'utm_medium', 'utm_content', 
                                        'utm_source', 'utm_term', 'gaId', 'yandexClientID', 'sourceGlobal']]
dr = pd.merge(left=dr, right=dc[dc['clickId'].notnull()].drop_duplicates('clickId'), on='clickId', how='left')

cre = cre_get(es, since)
dr['score'] = dr['userId_x'].map(cre).fillna(0).astype(int)

dr.to_pickle(f'reports/{datetime.strftime(since, "%B")}_claimReport.pickle.gz')
dc.to_pickle(f'reports/{datetime.strftime(since, "%B")}_claims.pickle.gz')

def AR_count(score, productType, status):
    koef = 1
    check = [2200, 1700, 1100, 5500]
    ARlist = [[0.00499500499500499, 0.00305343511450382, 0.0714285714285714, 0.0], 
              [0.00380818053596615, 0.00871508379888268, 0.0768345323741007, 0.0], 
              [0.00685378590078329, 0.0136418349605457, 0.0846340617412418, 0.0053475935828877], 
              [0.0111028867505552, 0.0255610972568579, 0.0980751604032997, 0.0116279069767442], 
              [0.00938967136150235, 0.0349985043374215, 0.126042632066728, 0.0], 
              [0.0109689213893967, 0.0407338308457712, 0.140204271123491, 0.0], 
              [0.0141884222474461, 0.0514050719671008, 0.113486842105263, 0.0188679245283019], 
              [0.0150753768844221, 0.0687361419068736, 0.0743243243243243, 0.0], 
              [0.00943396226415094, 0.0415430267062315, 0.0892857142857143, 0.0],
              [0.00943396226415094, 0.0415430267062315, 0.0892857142857143, 0.0]]
    
    if score is np.nan or status in ['NEW', 'ERROR']:
        return 0
    elif score > 10 and productType == 'CASH_LOAN':
        return round(ARlist[int(round(score, -1)/10)-1][0]*check[0]*koef, 2)
    elif score > 10 and productType == 'CREDIT_CARD':
        return round(ARlist[int(round(score, -1)/10)-1][1]*check[1]*koef, 2)
    elif score > 10 and productType == 'LOAN':
#         print(score, productType, status)
        return round(ARlist[int(round(score, -1)/10)-1][2]*check[2]*koef, 2)
    elif score > 10 and productType == 'REFINANCE':
        return round(ARlist[int(round(score, -1)/10)-1][3]*check[3]*koef, 2)
    else:
        return round(ARlist[int(round(score, -1)/10)-1][0]*check[0]*koef, 2)

dr['createdAt'] = pd.to_datetime(dr['createdAt'])
# dr['payout'] = dr['payout'].astype(float)
# dr['payout'] = dr.apply(lambda row: 3800 if row['partner']=='Альфа-Банк' 
#                         else row['payout'], axis=1)
# dr['payout'] = dr.apply(lambda row: 2550 if row['partner']=='Kredito24' 
#                         else row['payout'], axis=1)
# dr['productType'] = dr['product'].apply(lambda x: x['productType'])


dr['source'] = dr.apply(lambda row: 'Металинвест Cash' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)

#dr = dr[dr['status_x']!='NEW']
dr['score'] = dr['score'].apply(lambda x: x+75 if x==0 else x)
dr['whence']=dr.apply(lambda row: row['whence_y'] if pd.notnull(row['whence_y']) else row['whence_x'],axis=1)
dr['source']=dr.apply(lambda row: row['utm_source_y'] if pd.notnull(row['utm_source_y']) else row['whence'],axis=1)
dr['medium']=dr.apply(lambda row: row['utm_medium_y'] if pd.notnull(row['utm_medium_y']) else row['whence'],axis=1)

#sms = dr.groupby('source').agg({'_id':'count'}).to_excel('sms.xlsx')

dr['source']=dr.apply(lambda row: row['utm_source_y'] if 
                      pd.notnull(row['utm_source_y']) else row['whence'],axis=1)
dr['source']=dr.apply(lambda row: row['sourceGlobal'] if 
                      pd.notnull(row['sourceGlobal']) else row['source'],axis=1)
dr['source']=dr.apply(lambda row: 'Город Тройка' if 
                      row['utm_content']=='gorodtroika' else row['source'],axis=1)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Bankiros.ru ссылки' if 
                                  x in ['upside_mainfin', 'upside'] or x[:8]=='bankiros' else x)
dr['source'] = dr['source'].apply(lambda x: 'Vbr.ru ссылки' if 
                                  x in ['vbrru', 'upside_vbr','vbrr'] else x)
dr['source'] = dr['source'].apply(lambda x: 'Email' if 
                                 x[:5]=='email' or x[:7]=='TRIGGER' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x[:3]=='sms' else x)
dr['source'] = dr['source'].apply(lambda x: 'SMS рекламные' if 
                                   x in ['tele2TommorowWON',
                                         'tele2Tommorow',
                                         'tele2FourDays',
                                         'tele2TOP'] or x[:5]=='t2sms' or x =='lostBasket' else x)
dr['source'] = dr['source'].apply(lambda x: 'Admitad' if 
                                   x[:7]=='admitad' else x)
dr['source'] = dr['source'].apply(lambda x: 'Adv.Cake' if 
                                   x[:7]=='advcake' else x)
dr['source'] = dr['source'].apply(lambda x: 'Leadgid' if 
                                   x[:7]=='leadgid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Rubid' if 
                                   x[:5]=='rubid' else x)
dr['source'] = dr['source'].apply(lambda x: 'Cpahub' if 
                                   x[:6]=='cpahub' or x[:6]=='Cpahub' or x== 'get4click' 
                                  or x[:4]=='guru' or x[:9]=='guruleads' or x[:4]=='city' else x)
dr['source'] = dr['source'].apply(lambda x: 'Перфоманс через Артикс' if 
                                   x[:3]=='ais' else x)
dr['source'] = dr['source'].apply(lambda x: 'Tele2 // Микрозаймы' if 
                                   x[:4]=='tele' else x)
dr['source'] = dr['source'].apply(lambda x: 'Tele2 // Кредитные Карты' if 
                                   x[:5]=='t2_kk' else x)
dr['source'] = dr['source'].apply(lambda x: 'Виджеты' if 
                                   x[:10]=='substitute' else x)
dr['source'] = dr['source'].apply(lambda x: 'Браузерные пуши (web)' if 
                                   x[:4]=='push' else x)
dr['source'] = dr['source'].apply(lambda x: 'Exbico' if 
                                   x[:6]=='exbico' else x)
dr['source'] = dr['source'].apply(lambda x: 'Ссылочная масса' if 
                                   x[:8]=='showcase' else x)
dr['source'] = dr['source'].apply(lambda x: 'Статьи на сайте' if 
                                   x[:12]=='content-text' else x)
dr['source'] = dr['source'].apply(lambda x: 'МТС \\ баннер + СМС' if 
                                   x[:2]=='mt' or x[:6]=='stream' else x)
dr['source'] = dr['source'].apply(lambda x: 'Brobank.ru' if 
                                   x[:7]=='brobank' else x)
# dr['source'] = dr['source'].apply(lambda x: 'Крупный партнер' if 
#                                    x[:8]=='banki.ru' else x)            
dr['source'] = dr['source'].apply(lambda x: 'Тестирование гипотез' if 
                                  x[:6]=='yandex' or x[:6]=='google' else x)
dr['source']=dr.apply(lambda row: 'Тестирование гипотез' if 
                      row['utm_source_y']=='yandex' or row['utm_source_y']=='google' else row['source'],axis=1)
dr['source'] = dr.apply(lambda row: 'Крупный партнер (1)' if row['partner']=='banki.ru' 
                        else row['source'], axis=1)

dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5e9846be79dd27000141f476' 
                        else row['source'], axis=1)
dr['source'] = dr.apply(lambda row: 'Металлинвест \\ потребы' if row['productId_x']=='5ef4473acf22a90001010445' 
                        else row['source'], axis=1)


def get_kibana_reg(es, since,till):
    gte = (str(datetime.combine(since, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]
    lt = (str(datetime.combine(till, datetime.min.time()).timestamp()).replace(".", "")+"0000000")[:13]

    payload = {"size": 1000,
"query":{"bool":{"must":[{"match_all":{}},
                         {"match_phrase":{"eventType.keyword":"REGISTRATION"}},
                         {"range":{"timestamp":{"gte":gte,"lt":lt,"format":"epoch_millis"}}}]}}}
    
    d = es.search(index='events', doc_type='event', body=payload, scroll='120m', request_timeout=300)
    sleep(0.5)
    doc_count = d['hits']['total']
    hits = d['hits']['hits']
    print(doc_count, end ="; ", flush=True)
    res = []
    for i in range(1, int(doc_count/1000)+1):
        scroll = d['_scroll_id']
        d = es.scroll(scroll_id = scroll, scroll = '120m', request_timeout=300)
        hits = hits + d['hits']['hits']
        print(doc_count-1000*i, end ="; ", flush=True)
        sleep(0.5)

    dr = pd.DataFrame([hit['_source'] for hit in hits])
    dr.params.apply(lambda x: x.keys())
    for col in ['gaId', 'userId', 'utm_campaign', 'utm_medium', 'utm_source', 'utm_term', 'productType',
                'yandexClientID','currentPage',
#                 'yandexClientID', 'productId', 'whence', 'transitionId', 'partnerName', #'payout', 'status',
#                   'transaction_id', 'clickId', 
                'sourceGlobal']:
        dr[col] = dr['params'].apply(lambda x: x[col] if col in x.keys() else np.nan)

    dr['time'] = dr['timestamp'].apply(lambda x: datetime.strptime(x[:19], '%Y-%m-%dT%H:%M:%S'))
    dr['dt_reg'] = dr['time'].apply(lambda x: (x + timedelta(seconds=3*3600)).date())
    return dr

regs = get_kibana_reg(es, since,till)

# dr['utm_source_x'].value_counts()

regs['currentPage1'] = regs['currentPage'].apply(lambda x: x[:5] if pd.notnull(x) else np.nan)
regs['currentPage2'] = regs['currentPage'].apply(lambda x: x[:9] if pd.notnull(x) else np.nan)

dr=pd.merge(left=dr, right=regs, how='left', on='userId')

# odin = dr[dr['dt_reg'].notnull()]
# odin=odin[odin['product_type']=='LOAN']
# odin['userId'].nunique()

dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Android_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Huawei_mobile' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение' if row['sourceGlobal_y']=='Apple_mobile' 
                        else row['source_x'], axis=1)

dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage1']=='/news' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Статьи на сайте' if row['currentPage2']=='/articles' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Органический трафик' if row['source_x']=='Ссылочная масса' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'New (2Gis)' if row['source_x']=='2gis' 
                        else row['source_x'], axis=1)
dr['source_x'] = dr.apply(lambda row: 'Мобильное приложение 1' if row['source_x']=='Мобильное приложение' 
                        else row['source_x'], axis=1)
# Костыль

dr['status_x'] = dr.apply(lambda x: 'APPROVED' if
                        pd.notnull(x['status_y']) and x['status_y'] in(['APPROVED', 'approved', 'ISSUED']) 
                          else x['status_x'], axis=1)
                          


10986


INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/events/event/_search?scroll=120m [status:200 request:0.389s]


117276; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.338s]


107276; 

...

2635; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.033s]


1635; 

INFO:elasticsearch:GET http://elasticsearch-st-discovery.creditmarkt-bigdata.svc.cluster.local:9200/_search/scroll?scroll=120m [status:200 request:0.022s]


635; 

In [20]:
dr = dr[dr['source_x']=='Tele2 // Кредитные Карты']
dr['dt'] = pd.to_datetime(dr['dt'])
dr['date'] = dr['dt']
dr['income_group_first'] = dr['date'].apply(income_group)
df = dr.groupby('income_group_first',as_index=False).agg({'userId':'nunique'})
df.to_excel('Tele2 KK/Теле2_з.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1ZNM9Uscoyr6MQ02xOHJKzMO1GGzKDBuaFwwcW_EMnfU')
worksheet = sh.get_worksheet(15) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 KK/Теле2_з.xlsx')
set_with_dataframe(worksheet, df1)



In [21]:
dr.head(5)

,_id,clickId,type,createdAt,updatedAt,status_x,borrower,partner,product,statusFields,adapterName,whence_x,id_x,dt,productId_x,product_type,product_name,userId_x,phone,requestId,indexName_x,fallbackResponse_x,json_x,id_y,eventType_x,source_x,params_x,timestamp_x,gaId_x,userId_y,utm_campaign_x,utm_medium_x,utm_source_x,utm_term_x,productType_x,yandexClientID_x,productId_y,whence_y,transitionId,partnerName,payout,status_y,transaction_id,externalId,orderId,transactionId,time_x,date,userId,utm_campaign_y,utm_medium_y,utm_content,utm_source_y,utm_term_y,gaId_y,yandexClientID_y,sourceGlobal_x,score,whence,medium,indexName_y,fallbackResponse_y,json_y,id,eventType_y,source_y,params_y,timestamp_y,gaId,utm_campaign,utm_medium,utm_source,utm_term,productType_y,yandexClientID,currentPage,sourceGlobal_y,time_y,dt_reg,currentPage1,currentPage2,income_group_first
2420,60faed03af4ba60001929b28,06cbf745-39f6-406f-8547-fe81f2d42cb9,LINK,2021-07-23 16:23:31.217,2021-09-20 16:34:46.598,REJECTED,"{'_id': None, 'userId': '60faebdd357065000157962f', 'fullName': 'Карповец Наталья Александровна', 'email': 'lapka.25@list.ru', 'phone': '79009112994'}",МТС Банк,"{'_id': 5bffbf3eb340050001a7f445, 'name': 'Кредитная карта MTS CASHBACK', 'productType': 'CREDIT_CARD', 'productName': 'Кредитная карта'}","{'LINK': None, 'NOTE': '9218'}",LEADGID_PARTNER,showcase,60faed03af4ba60001929b28,2021-07-23,5bffbf3eb340050001a7f445,CREDIT_CARD,Кредитная карта MTS CASHBACK,60faebdd357065000157962f,79009112994,NaN,NaN,NaN,NaN,NaN,NaN,Tele2 // Кредитные Карты,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2021-07-23,60faebdd357065000157962f,podborcard,NaN,NaN,t2_kk,grace_period,GA1.2.1997066049.1627057419,1627057419723411121,NaN,1,showcase,showcase,None,False,"{""id"":""42a3d9c2-6d9e-4a1d-8388-660475569607"",""eventType"":""REGISTRATION"",""timestamp"":""2021-07-23T16:19:57.977Z"",""source"":""public"",""indexName"":null,""params"":{""gaId"":""GA1.2.1997066049.1627057419"",""sessionId"":""4dded83f-b469-4d6d-876f-2d6b5e39a830"",""yandexClientID"":""1627057419723411121"",""currentPage"":""/entry/registration/card"",""nextPage"":""/entry/registration/card"",""userId"":""60faebdd357065000157962f"",""utm_source"":""t2_kk"",""utm_campaign"":""podborcard"",""productType"":""cash"",""utm_term"":""grace_period"",""ipAddress"":""176.59.2.173""},""fallbackResponse"":false}",42a3d9c2-6d9e-4a1d-8388-660475569607,REGISTRATION,public,"{'utm_term': 'grace_period', 'gaId': 'GA1.2.1997066049.1627057419', 'utm_campaign': 'podborcard', 'yandexClientID': '1627057419723411121', 'nextPage': '/entry/registration/card', 'ipAddress': '176.59.2.173', 'sessionId': '4dded83f-b469-4d6d-876f-2d6b5e39a830', 'currentPage': '/entry/registration/card', 'userId': '60faebdd357065000157962f', 'productType': 'cash', 'utm_source': 't2_kk'}",2021-07-23T16:19:57.977Z,GA1.2.1997066049.1627057419,podborcard,NaN,t2_kk,grace_period,cash,1627057419723411121,/entry/registration/card,NaN,2021-07-23 16:19:57,2021-07-23,/entr,/entry/re,Июльская рассылка
2422,60faedc9198105000123f24e,13ce1c9a-fc48-43bf-8129-68bce50c8105,LINK,2021-07-23 16:26:49.728,2021-08-22 16:34:22.091,REJECTED,"{'_id': None, 'userId': '60faecf9d42c470001d9047c', 'fullName': 'Захаров Илья Леонидович', 'email': 'zaxar123111@mail.ru', 'phone': '79045968913'}",Газпромбанк,"{'_id': 5d1dc175f1bf540001090a84, 'name': '«Удобная карта» от Газпромбанка', 'productType': 'CREDIT_CARD', 'productName': 'Кредитная карта'}","{'LINK': None, 'NOTE': '9944'}",NaN,showcase,60faedc9198105000123f24e,2021-07-23,5d1dc175f1bf540001090a84,CREDIT_CARD,«Удобная карта» от Газпромбанка,60faecf9d42c470001d9047c,79045968913,NaN,NaN,NaN,NaN,NaN,NaN,Tele2 // Кредитные Карты,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,2021-07-23,60faecf9d42c470001d9047c,podborcard,NaN,NaN,t2_kk,grace_period,GA1.2.1223167279.1627057386,1627057387357266407,NaN,1,showcase,showcase,None,False,"{""id"":""1eebaf7c-6ed8-424c-b690-1c8b797

In [22]:
df3 = dr.groupby(['income_group_first','product_type'],as_index=False).agg({'userId_x':'count'})
df3

,income_group_first,product_type,userId_x
0,Июльская рассылка,CASH_LOAN,5
1,Июльская рассылка,CREDIT_CARD,43
2,Июльская рассылка,LOAN,6
3,Октябрьская рассылка,CREDIT_CARD,87
4,Октябрьская рассылка,LOAN,1
5,не определено,CREDIT_CARD,2
